# TF-IDF Search Using Cosine Similarity

In [11]:
import pandas as pd
import numpy as np
import spacy
from collections import Counter
import math
from sklearn.metrics.pairwise import cosine_similarity

In [12]:
text_df = pd.read_json("processed_text.json")
text_df.head()

,title,text,url,processed_text
0,Pandemic,"A pandemic (from Greek πᾶν, pan, ""all"" and δῆμ...",https://en.wikipedia.org/wiki/Pandemic,"[pandemic, greek, pan, demo, people, epidemic,..."
1,Epidemiology of HIV/AIDS,"HIV/AIDS, or Human Immunodeficiency Virus, is ...",https://en.wikipedia.org/wiki/Epidemiology_of_...,"[hiv, aid, human, immunodeficiency, virus, con..."
2,Antonine Plague,"The Antonine Plague of 165 to 180 AD, also kno...",https://en.wikipedia.org/wiki/Antonine_Plague,"[plague, ad, know, plague, galen, galen, physi..."
3,Basic reproduction number,"In epidemiology, the basic reproduction number...",https://en.wikipedia.org/wiki/Basic_reproducti...,"[epidemiology, basic, reproduction, number, ba..."
4,Bills of mortality,Bills of mortality were the weekly mortality s...,https://en.wikipedia.org/wiki/Bills_of_mortality,"[bill, mortality, weekly, mortality, statistic..."


## Term Frequency

**The number of times a word appears in a document divided by the total times it appears in the corpus. Every document has its own term frequency.**

In [13]:
def computeTF(single_doc, docs):
    """Computes the Term Frequency given the tokenized text string doc and a list of tokenized documents all_docs. Returns a complete vocabulary of all_docs as a  dictionary of words with number of word appearances in doc divided by number of word appearances in all_docs

    Args:
        doc (list): A preprocessed text string tokenized into a list of words
        all_docs (list): A list of tokenized text strings now represented as word lists
    """
    doc_counts = Counter(single_doc)
    corpus = [word for sublist in docs for word in sublist]
    corpus_counts = Counter(corpus)
    tf_dict = dict.fromkeys(corpus, 0)
    for key in doc_counts.keys():
       tf_dict[key] = doc_counts[key]/corpus_counts[key]
      
    return(tf_dict)

## Inverse Document Frequency

**The number of documents divided by the number of documents that contain the word w. Inverse data frequency determines the weight of rare words across all documents in the corpus.**

In [14]:
def computeIDF(docs):    
    """ Accepts a list of documents and returns the number of total documents divided by the number of documents containing a given word for each word
    Args:
        doc (list): A list of tokenized text strings now represented as word lists
    """
    N = len(docs)
    corpus = [word for sublist in docs for word in sublist]
    idfDict = dict.fromkeys(corpus, 0)
    for document in docs:
        for word in list(set(document)):
            idfDict[word] += 1
    
    for word, val in idfDict.items():
        idfDict[word] = N/float(val)
    return idfDict

## Putting TF and IDF together

In [40]:
# Compute Inverse Document Frequency
def build_tf_idf(doc, docs):
    """Given a preprocessed string and a list of documents returns the TF-IDF vectorization

    Args:

        doc (list): A tokenized list of words
        docs (list): A list of tokenized documents that make up entire vocab
    """

    tf_idf_dictionary = dict()
    tf_dict = computeTF(doc, docs)
    idf_dict = computeIDF(docs)

    for word, val in tf_dict.items():
        tf_idf_dictionary[word] = (tf_dict[word]*idf_dict[word])
    
    return tf_idf_dictionary
        


In [41]:
text_df["tfidf_vector"] = text_df.processed_text.apply(build_tf_idf, docs=text_df.processed_text)
text_df.head()


,title,text,url,processed_text,tfidf_vector
0,Pandemic,"A pandemic (from Greek πᾶν, pan, ""all"" and δῆμ...",https://en.wikipedia.org/wiki/Pandemic,"[pandemic, greek, pan, demo, people, epidemic,...","{'pandemic': 0.1814556331006979, 'greek': 26.0..."
1,Epidemiology of HIV/AIDS,"HIV/AIDS, or Human Immunodeficiency Virus, is ...",https://en.wikipedia.org/wiki/Epidemiology_of_...,"[hiv, aid, human, immunodeficiency, virus, con...","{'pandemic': 0.051844466600199396, 'greek': 0...."
2,Antonine Plague,"The Antonine Plague of 165 to 180 AD, also kno...",https://en.wikipedia.org/wiki/Antonine_Plague,"[plague, ad, know, plague, galen, galen, physi...","{'pandemic': 0.051844466600199396, 'greek': 0...."
3,Basic reproduction number,"In epidemiology, the basic reproduction number...",https://en.wikipedia.org/wiki/Basic_reproducti...,"[epidemiology, basic, reproduction, number, ba...","{'pandemic': 0.0, 'greek': 0.0, 'pan': 0.0, 'd..."
4,Bills of mortality,Bills of mortality were the weekly mortality s...,https://en.wikipedia.org/wiki/Bills_of_mortality,"[bill, mortality, weekly, mortality, statistic...","{'pandemic': 0.0, 'greek': 0.0, 'pan': 0.0, 'd..."


In [42]:
text_tf_idf = text_df.to_json()

with open('processed_text_tf_idf.json', 'w') as outfile:
    outfile.write(text_tf_idf)

## Create a Search Function using cosine similarity

In [8]:
def make_tokens(text):
    """ Accepts any list of words and returns a tokenized list

    Args:
        text (string):

    Returns:
        list: Tokenizeed list of lemmatized words
    """
    nlp = spacy.load("en_core_web_lg") # Initialize the vocabulary
    doc = nlp(text.lower())
    filtered_sentence =[] 
    for word in doc:
        lexeme = nlp.vocab[str(word)]
        if lexeme.is_stop == False and lexeme.is_punct == False and lexeme.is_oov == False:
            filtered_sentence.append(word.lemma_) 
    return filtered_sentence

In [110]:
def get_cosine_similarities(string, titles, documents):
    """Accepts a string, list of title, documents, and a tfidf vector returns a dictionary of title with cosine similarity score ordered by most similar to least similar

    Args:
        string (string): Any string of words
        titles (list): A list of document titles associated with the  vectorized documents given in the tfidf_vector argument
        documents (list): A list of preprocessed document token vectors to build vocabularity for tf-idf computation
    """
    tokenized_string = make_tokens(string)
    input_tfidf = np.array([list(build_tf_idf(tokenized_string, documents).values())])
    document_tf_idf = pd.Series([np.array([list(dictionary.values())]) for dictionary in text_df.processed_text.apply(build_tf_idf, docs = text_df.processed_text)])
    similarity_scores = document_tf_idf.apply(cosine_similarity, Y=input_tfidf)
    similarity_scores = pd.Series([value.item(0) for value in similarity_scores])
    scores_df = pd.concat([titles, similarity_scores.rename("similarity_score")], axis=1)
    sorted_scores = scores_df.sort_values(by = "similarity_score", ascending=False)
    sorted_scores_dict = sorted_scores.set_index("title")['similarity_score'].to_dict()
    return sorted_scores_dict

In [111]:
get_cosine_similarities("Current Covid-19 Pandemic", text_df.title, text_df.processed_text)

{'Pandemic': 0.06806871147923645,
 'Basic reproduction number': 0.034561731594380724,
 'Pandemic Severity Assessment Framework': 8.96558888535273e-06,
 'Pandemic prevention': 8.351490595603745e-06,
 'Spanish flu': 4.417932748624734e-06,
 'Pandemic severity index': 3.7607663165594926e-06,
 'Crimson Contagion': 2.670011341117137e-06,
 'Swine influenza': 2.4215663904203813e-06,
 'COVID-19 pandemic': 2.184591947733632e-06,
 'Plague of Cyprian': 1.7559632743903951e-06,
 'PREDICT (USAID)': 1.739019649931866e-06,
 '1929–1930 psittacosis pandemic': 1.462630521558065e-06,
 'Antonine Plague': 1.2528671116264168e-06,
 'Epidemiology of HIV/AIDS': 1.240976796349597e-06,
 'Science diplomacy and pandemics': 7.48822242187819e-07,
 'Unified Victim Identification System': 6.016808544401113e-07,
 'HIV/AIDS': 5.000858286510558e-07,
 'Cholera': 4.664497798098712e-07,
 'Bills of mortality': 0.0,
 'HIV/AIDS in Yunnan': 0.0,
 'Event 201': 0.0,
 'Disease X': 0.0,
 'Superspreader': 0.0,
 'Targeted immunization 